# Creating and Valuing a CDS Contract vs Markit

Example CDS Valuation and comparison with market standard ISDA model on Markit website

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.300 - This build:  04 Jul 2023 at 11:05 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Creating a CDS Contract

This based on an example from Markit's CDS calculator website https://www.markit.com/markit.jsp?jsppage=pv.jsp

In [3]:
valuation_date = Date(21, 11, 2019)
settlement_date = valuation_date.add_days(1)
maturity_date = Date(20, 12, 2024)
cdsCoupon = 0.050
notional = ONE_MILLION
long_protection = True
tradeDate = Date(9, 8, 2019)

In [4]:
cds_contract = CDS(settlement_date, maturity_date, cdsCoupon, notional, long_protection)

In [5]:
print(cds_contract)

OBJECT TYPE: CDS
STEP-IN DATE: 22-NOV-2019
MATURITY: 20-DEC-2024
NOTIONAL: 1000000
RUNNING COUPON: 500.0bp
DAYCOUNT: DayCountTypes.ACT_360
FREQUENCY: FrequencyTypes.QUARTERLY
CALENDAR: CalendarTypes.WEEKEND
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD
ACCRUED DAYS: 63.0
PAYMENT_DATE, YEAR_FRAC, ACCRUAL_START, ACCRUAL_END, FLOW
20-DEC-2019,     0.252778, 20-SEP-2019, 19-DEC-2019, 12638.888889
20-MAR-2020,     0.252778, 20-DEC-2019, 19-MAR-2020, 12638.888889
22-JUN-2020,     0.261111, 20-MAR-2020, 21-JUN-2020, 13055.555556
21-SEP-2020,     0.252778, 22-JUN-2020, 20-SEP-2020, 12638.888889
21-DEC-2020,     0.252778, 21-SEP-2020, 20-DEC-2020, 12638.888889
22-MAR-2021,     0.252778, 21-DEC-2020, 21-MAR-2021, 12638.888889
21-JUN-2021,     0.252778, 22-MAR-2021, 20-JUN-2021, 12638.888889
20-SEP-2021,     0.252778, 21-JUN-2021, 19-SEP-2021, 12638.888889
20-DEC-2021,     0.252778, 20-SEP-2021, 19-DEC-2021, 12638.888889
21-MAR-2022,     0.252778, 20-DEC-2021, 20-

## Build Ibor Curve

In [6]:
dcType = DayCountTypes.ACT_360
depo1 = IborDeposit(settlement_date, "1M", 0.017156, dcType)
depo2 = IborDeposit(settlement_date, "2M", 0.018335, dcType)
depo3 = IborDeposit(settlement_date, "3M", 0.018988, dcType)
depo4 = IborDeposit(settlement_date, "6M", 0.018911, dcType)
depo5 = IborDeposit(settlement_date, "12M", 0.019093, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

swapType = SwapTypes.PAY
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swap1 = IborSwap(settlement_date,"2Y",swapType,0.015630,fixedFreq,dcType)
swap2 = IborSwap(settlement_date,"3Y",swapType,0.015140,fixedFreq,dcType)
swap3 = IborSwap(settlement_date,"4Y",swapType,0.015065,fixedFreq,dcType)
swap4 = IborSwap(settlement_date,"5Y",swapType,0.015140,fixedFreq,dcType)
swap5 = IborSwap(settlement_date,"6Y",swapType,0.015270,fixedFreq,dcType)
swap6 = IborSwap(settlement_date,"7Y",swapType,0.015470,fixedFreq,dcType)
swap7 = IborSwap(settlement_date,"8Y",swapType,0.015720,fixedFreq,dcType)
swap8 = IborSwap(settlement_date,"9Y",swapType,0.016000,fixedFreq,dcType)
swap9 = IborSwap(settlement_date,"10Y",swapType,0.016285,fixedFreq,dcType)
swap10 = IborSwap(settlement_date,"12Y",swapType,0.01670,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

# Build a CDS Curve

In [7]:
cdsFlatSpread = 0.0100

In [8]:
cds1 = CDS(settlement_date, "1Y", cdsFlatSpread)
cds2 = CDS(settlement_date, "2Y", cdsFlatSpread)
cds3 = CDS(settlement_date, "3Y", cdsFlatSpread)
cds4 = CDS(settlement_date, "4Y", cdsFlatSpread)
cds5 = CDS(settlement_date, "5Y", cdsFlatSpread)
cds6 = CDS(settlement_date, "7Y", cdsFlatSpread)
cds7 = CDS(settlement_date, "10Y", cdsFlatSpread)
cds8 = CDS(settlement_date, "15Y", cdsFlatSpread)

In [9]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [10]:
recovery_rate = 0.40

In [11]:
issuer_curve = CDSCurve(valuation_date, cdss, libor_curve, recovery_rate)

# Valuation Results

In [12]:
spd = cds_contract.par_spread(valuation_date, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  100.00000 bp


In [13]:
v = cds_contract.value(valuation_date, issuer_curve, recovery_rate)
dirty_pv = v['dirty_pv']
clean_pv = v['clean_pv']

print("DIRTY VALUE %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

FULL VALUE    -198532.56
CLEAN VALUE   -189782.56


MARKIT CALCULATE -198,551 for the DIRTY VALUE

In [14]:
cleanp = cds_contract.clean_price(settlement_date, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   118.979987


MARKIT CALCULATE 118.98%

In [15]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 63.0


MARKIT CALCULATE 63

In [16]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON -8750.0


MARKIT CALCULATE 8750

In [17]:
prot_pv = cds_contract.protection_leg_pv(settlement_date, issuer_curve, recovery_rate)
print("PROTECTION_PV", prot_pv)

PROTECTION_PV 47449.853178301884


In [18]:
premPV = cds_contract.premium_leg_pv(settlement_date, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 245999.71571231817


## Risk Measures

In [19]:
credit_dv01 = cds_contract.credit_dv01(settlement_date, issuer_curve, recovery_rate)
print("Credit DV01  %12.2f"% credit_dv01)

Credit DV01        553.53


MARKIT FOUND 554

In [20]:
interest_dv01 = cds_contract.interest_dv01(settlement_date, issuer_curve, recovery_rate)
print("Interest DV01  %12.2f"% interest_dv01)

Interest DV01         48.82


MARKIT FOUND 49

## Full Analysis 

In [23]:
cds_contract.print_flows(valuation_date, issuer_curve)

PAYMENT_DATE      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
    20-DEC-2019   0.252778     12638.89     0.998620     0.998658     12604.51
    20-MAR-2020   0.252778     12638.89     0.993728     0.994459     12490.03
    22-JUN-2020   0.261111     13055.56     0.988864     0.990141     12782.88
    21-SEP-2020   0.252778     12638.89     0.984127     0.985978     12263.86
    21-DEC-2020   0.252778     12638.89     0.979968     0.981833     12160.69
    22-MAR-2021   0.252778     12638.89     0.977072     0.977711     12073.86
    21-JUN-2021   0.252778     12638.89     0.974185     0.973607     11987.65
    20-SEP-2021   0.252778     12638.89     0.971306     0.969521     11902.05
    20-DEC-2021   0.252778     12638.89     0.968270     0.965451     11815.05
    21-MAR-2022   0.252778     12638.89     0.964873     0.961400     11724.19
    20-JUN-2022   0.252778     12638.89     0.961488     0.957365     11634.03
    20-SEP-2022   0.255556     12777.78     0.958078   